# An introduction to Numpy

In [ ]:
%run ../00_AdvancedPythonConcepts/talktools.py

Some useful references:

- The [numpy tutorial at scipy.org](http://wiki.scipy.org/Tentative_NumPy_Tutorial) is a handy read for the key ideas around using Numpy.

- The [Numpy/Scipy documentation](http://docs.scipy.org/doc).

- A set of [Lectures on scientific computing with Python](https://github.com/jrjohansson/scientific-python-lectures#online-read-only-versions), written as a collection of Notebooks.

- The [Python Scientific Lecture Notes](http://scipy-lectures.github.io) also contain lots of useful materials on today's topics.

- If you come from an IDL background, this [IDL-to-Numpy guide](https://www.cfa.harvard.edu/~jbattat/computer/python/science/idl-numpy.html) will be useful.

- Similarly, [for Matlab users](http://www.scipy.org/NumPy_for_Matlab_Users).

(c) J. Bloom UC Berkeley, 2016, 2018 

## The `ndarray` object

The basic building block of the numpy package is the ndarray: a multidimensional, homogeneous array of fixed-size items.

An associated data-type object describes the format of
each element in the array (its byte-order, how many bytes it occupies in
memory, whether it is an integer, a floating point number, or something
else, etc.)
<p>
<center>
<img src=http://docs.scipy.org/doc/numpy/_images/threefundamental.png></img>
</center>

New instances of the ndarray class are never (rarely) created directly, but instead through a method that returns one.

In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np

In [ ]:
a = np.array([1, 2, 3])
a

In [ ]:
b = np.ones((3,2))
b

In [ ]:
c = np.empty((4,3,2))
c

In [ ]:
d = np.zeros((3,2))
d

Is it worth using `np.empty` for performance reasons? Use `%timeit` to figure it out!

In [ ]:
for s in [1e3, 1e5, 1e7]:
    size = int(s)
    print('Timing array creation with:', size, 'elements')
    print("Zeros:")
    %timeit np.zeros(size)
    print("Empty:")
    %timeit np.empty(size)
    print("*"*20)

I very frequently use linspace and logspace to create regularly sampled 1 dimensional ndarrays.  The syntax is `linspace(start, end, number_of_points)`:

In [ ]:
np.linspace(1,10,5)

In [ ]:
np.logspace(1,2,5)

## Quick recap of array access

ndarrays can be sliced, indexed, and iterated over much like lists.

In [ ]:
a = np.arange(10)**3

In [ ]:
a

In [ ]:
a[2]

In [ ]:
a[2:5]

In [ ]:
a[:6:2] = -1000
a

In [ ]:
a[::-1]

In [ ]:
for i in a:
    print(i**(1/3.), end=" ... ")

A small note: what is this `nan` business?

In [ ]:
b=a**(-1/3.) ; print(b[0])

In [ ]:
b[0] == np.nan

In [ ]:
np.isnan(b[0])

## Reading and writing

Recall from the bootcamp, that:

- `np.save`: write an array to a binary file with `.npy` extension.
- `np.savez`: write several arrays to a binary file with `.npz` extension.
- `np.load`: read one or more arrays from a `.npy` or `.npz` file.
- `np.savetxt`: writes an array to a text file.
- `np.loadtxt`: reads an array from a text file.

Furthermore, the `scipy.io` module has even more reader/writers for ndarrays into other formats.

## Structured Arrays

ndarrays can be composed of (almost) any data type.  The dtype attribute specifies the structure of the stored data. Simple dtypes like `int` and `np.float64` can be used to describe simple numerical types, but numpy can represent elements that have multiple fields each with its own type, naming each one of them.

The syntax is as follows (for more details [see the docs](http://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html)):

In [ ]:
# a list of (name, format) tuples.
dtype_descriptor = [ ('x', np.int32), 
                     ('y', np.float32), 
                     ('name', (np.string_, 10)) ] 
my_arr = np.zeros(2, dtype=dtype_descriptor )

In [ ]:
my_arr[:] = [(1,2.,'Hello'),(2,3.,"World")]
my_arr

In [ ]:
my_arr[0][0] = 3

In [ ]:
my_arr

Note that the above dtype descriptor can also be specified with the following syntax instead, which is fully equivalent in functionality:

In [ ]:
# A dict with keys 'names' and 'formats', and values respectively tuples with all
# names and all format descriptors. 
# Note that 'S10' is equivalent to (np.string_, 10) to describe a 10-character string
dtype_descriptor = dict(names = ('x', 'y', 'name'),
                        formats = (np.int32, np.float32, 'S10'))

In [ ]:
my_arr[1]

In [ ]:
y = my_arr['y']
y

In [ ]:
y[:] = 2*y
y

In [ ]:
my_arr

##   Copying and Referencing

The behavior of ndarrays is *different* to that of lists when copying and referencing is involved, and it's very important to understand these differences.

Let's begin by recalling that in Python, two variable names can point to the same object:

In [ ]:
lst1 = list(range(10))
print('lst1:', lst1)
lst2 = lst1
lst1 is lst2

In [ ]:
lst2[0] = -9999
lst1

But note that for Python lists, slicing creates a *copy* of the original:

In [ ]:
lst3 = lst1[::2]
print('lst1:', lst1)
print('lst3:', lst3)

This means that modifying `lst3` will *not* modify `lst1`:

In [ ]:
lst3[0] = 100
lst3[1] = 200
print('lst1:', lst1)
print('lst3:', lst3)

Now let's see what happens when we slice arrays:

In [ ]:
a1 = np.arange(10)
a2 = a1[::2]
print('a1', a1)
print('a2', a2)

But when we slice an array, we get a new array which is a *view* over the memory of the original one. This emans that changes to any one of them are seen by the other:

In [ ]:
a[0] = -99
print('a1', a1)
print('a2', a2)

In [ ]:
a2[1] = -9999
print('a1', a1)
print('a2', a2)

In [ ]:
a2.base is a1

In [ ]:
print('Data type                :', a1.dtype)
print('Total number of elements :', a1.size)
print('Number of dimensions     :', a1.ndim)
print('Shape (dimensionality)   :', a1.shape)
print('Memory used (in bytes)   :', a1.nbytes)

There are also many useful functions in numpy that operate on arrays, e.g.:

In [ ]:
print('Minimum and maximum             :', np.min(a1), np.max(a1))
print('Sum and product of all elements :', np.sum(a1), np.prod(a1))
print('Mean and standard deviation     :', np.mean(a1), np.std(a1))

For these methods, the above operations area all computed on all the elements of the array.  But for a multidimensional array, it's possible to do the computation along a single dimension, by passing the `axis` parameter; for example:

In [ ]:
a1 = a1.reshape((2,5))
print('For the following array:\n', a1)
print('The sum of elements along the rows is    :', np.sum(a1, axis=1))
print('The sum of elements along the columns is :', np.sum(a1, axis=0))

## The array `flags` field

This is a good time to look at the `flags` field of an ndarray, which tells us a lot of information about its internals:

In [ ]:
a1.flags

Let's compare that to `a2`:

In [ ]:
a2.flags

Each of these fields is accessible individually, which you can use in your own code:

In [ ]:
a2.flags.owndata

The `astype` method can be used to create another view of an existing array with a different dtype:

In [ ]:
print(a1)
print(a1.shape, a1.dtype)

In [ ]:
a3 = a1.astype(np.int8)
print(a3.shape, a3.dtype)

In [ ]:
print(a3)

If you want to create a separate copy of the data of an array (a "deep" copy in python parlance), use the copy method associated with the ndarray class.

In [ ]:
b = a1.copy()

In [ ]:
b is a1

In [ ]:
b.base is a1

In [ ]:
b[0] = 1234
b

In [ ]:
a1

## Numpy "fancy indexing"

Arrays can be indexed with other arrays instead of using simple integers or slice notation. This is known in the numpy literature as [fancy indexing](http://wiki.scipy.org/Tentative_NumPy_Tutorial#head-0dffc419afa7d77d51062d40d2d84143db8216c2), and the best way to wrap your head around it is to experiment with small examples yourself.

It is important to note that when fancy indexing is used, the resulting output is *always* a copy of the data. Fancy indexing never produces views.

In [ ]:
a = np.arange(12)**2                          # the first 12 square numbers
i = np.array( [ 1,1,3,8,5 ] )                 # an array of indices
a[i]                                       # the elements of a at the positions i

The shape of the index array is what determines the shape of the output! For example, if we index into `a` (which is 1-d) with a 2-d index array, we'll get a 2-d output shaped like the index:

In [ ]:
j = np.array( [ [ 3, 4], [ 9, 7 ] ] )
a[j]

We can use fancy indexing with multidimensional arrays:

In [ ]:
a = np.arange(12).reshape(3,4)
a

The arrays of indices for each dimension must have the same shape, and *this will be the shape of the output*.

In [ ]:
i = np.array( [ [0,1], [1,2] ] )  # our output will be shaped (2,2)
j = np.array( [ [2,1], [3,3] ] )

In [ ]:
a[i,j]

Fancy and 'plain' indexing can be mixed along different dimensions:

In [ ]:
a[i,2]

In [ ]:
b = a.ravel()

In [ ]:
b.flags.owndata

<div class="alert alert-info"> 
<font size=+1>in general, ndarrays methods which return ndarrays, return views.

Functions which operate on ndarrays, return new ndarrays</font>
</div>

## Finding parts of arrays that match conditions

Since conditional expressions return boolean arrays, and we know we can use arrays (as seen above) to index, we can use very simple syntax.

Let's imagine we want to find where the elements of `a` are not zero:

In [ ]:
a = np.array([1, 3, 0, -5, 0], float)

In [ ]:
a != 0

In [ ]:
a[a != 0]

The where method provides a convenient (though not always fast) way to search and extract individual elements of an ndarray.  The return value of where is ready to be used for fancy indexing:

In [ ]:
nz = np.where(a != 0)
nz

In [ ]:
a[nz]

In [ ]:
x = np.arange(9.).reshape(3, 3)
x

In [ ]:
x[np.where( x > 5 )]

For simple conditions selection, the standard boolean index syntax is more readable:

In [ ]:
x[x>5]

In [ ]:
x[(x>5) & (x < 7)]

But `where` can also be used to select elements and return instead an array, taking from either argument depending on if the condition is true or false:

In [ ]:
np.where(a != 0.0, a**2, a)  # if a!=0, return a*a, else return a

## Universal Functions

A universal function (or ufunc for short) is a function that operates on ndarrays in an
element-by-element fashion, supporting *array broadcasting*, *type casting*, and several other standard features. That is, a ufunc is a “*vectorized*” wrapper for a function that takes a fixed number of scalar inputs and produces a fixed number of scalar outputs. 

Examples include `add`, `subtract`, `multiply`, `exp`, `log`, `power`, `sin`, `cos` and `tan`.

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[2, 3], [4, 5]])

In [ ]:
a+b

In [ ]:
np.multiply(a,b)  # identical to a*b

In [ ]:
np.power(a,b)  # identical to a**b

ndarray objects have methods for (very) basic statistics.  

`scipy` and `statsmodels` provide high-power statistical functionality, and `scikit-learn` provides full-fledged machine learning tools.

In [ ]:
a = np.array([[1, 2], [3, 4]])

In [ ]:
np.mean(a), a.mean()

In [ ]:
np.mean(a, axis=0), np.mean(a, axis=1)

In [ ]:
np.std(a)

In [ ]:
np.average(range(1,11), weights=range(10,0,-1))

## Vectorizing with `numexpr`

ufuncs are much fast than for loops.  But the numexpr module provides even faster vectorization.

In [ ]:
a = np.arange(1e6)
b = np.arange(1e6)

In [ ]:
%timeit a**2 + b**2 + 2*a*b

In [ ]:
!conda install numexpr -y

In [ ]:
import numexpr as ne

In [ ]:
%timeit ne.evaluate("a**2 + b**2 + 2*a*b")

In [ ]:
ne.set_num_threads(2)

In [ ]:
%timeit ne.evaluate("a**2 + b**2 + 2*a*b")

## Random Sampling

The numpy.random module contains the most common probability distribution functions, as well as a random number generator.  scipy contains much more sophisticated probability and statistics modules.

In [ ]:
# <-- seed value, do not have to specify, but useful for reproducibility
rng = np.random.RandomState(0)
mu, sigma = 0, 0.1
s = rng.normal(mu, sigma, 1000)

Let's load matplotlib to visualize this

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

And now we can use the matplotlib `hist` function to plot our sampled data, along with the analytical formula for the underlying normal distribution:

In [ ]:
count, bins, ignored = plt.hist(s, 30, normed=True)
plt.plot(bins, np.exp( -(bins-mu)**2 / (2*sigma**2) ) / (sigma * np.sqrt(2*np.pi)), 
         color='r', lw=2.0, label=r'$\cal{N(\mu, \sigma)}$');
plt.legend(fontsize=15);

# Breakout session

1) Make an ndarray representing a 52 deck of cards (A, 2-10, J, Q, K + 4 suits) where each element represents a unique card including it’s numerical equivalent (e.g., A = 1, K = 13)

    >>> print(deck[11])
    ('Q', 'C', 12)

2) Drawing 5 cards randomly from a shuffled deck, what is probability of getting at least two cards of the same value (ie. a pair)? What is the probability of getting all five cards from the same suit (ie. a flush)?

*Hint:* `np.unique`, `np.random.shuffle`.

## Broadcasting

### The broadcasting rules

This broadcasting behavior is powerful, especially because when numpy broadcasts to create new dimensions or to 'stretch' existing ones, it doesn't replicate the data.  In the example above the operation is carried *as if* the 3 was a 1-d array with 3 in all of its entries, but no actual array was ever created.  This can save memory in cases when the arrays in question are large, with significant performance implications.

The general rule is: when operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward, creating dimensions of length 1 as needed. Two dimensions are considered compatible when

* they are equal or either is None or one
* either dimension is 1 or ``None``, or if dimensions are equal

If these conditions are not met, a `ValueError: frames are not aligned` exception is thrown, indicating that the arrays have incompatible shapes. The size of the resulting array is the maximum size along each dimension of the input arrays.

Examples below:

```
(9, 5)   (9, 5)   (9, 5)   (9, 1)
   ( )   (9, 1)   (   5)   (   5)
------   ------   ------   ------
(9, 5)   (9, 5)   (9, 5)   (9, 5)

```

<img src="broadcast_rougier.png"/>

Sketch from [Nicolas Rougier's NumPy tutorial](http://www.labri.fr/perso/nrougier/teaching/numpy/numpy.html)

### Visual illustration of broadcasting
<center>
<img src="numpy_broadcasting.svg" width=80%>
</center>

In [ ]:
x = np.arange(4)
xx = x.reshape(4,1)
y = np.ones(5)
z = np.ones((3,4))

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x + y

In [ ]:
xx.shape

In [ ]:
(xx + y).shape

In [ ]:
xx + y

In [ ]:
z.shape

In [ ]:
(x + z).shape

In [ ]:
x + z

For the full broadcasting rules, please see the official Numpy docs, which describe them in detail and with more complex examples.

Also see: [G-Node Summer School Advanced NumPy tutorial](https://github.com/stefanv/teaching/blob/master/2014_assp_split_numpy/numpy_advanced.ipynb)

As we mentioned before, Numpy ships with a full complement of mathematical functions that work on entire arrays, including logarithms, exponentials, trigonometric and hyperbolic trigonometric functions, etc.  Furthermore, scipy ships a rich special function library in the `scipy.special` module that includes Bessel, Airy, Fresnel, Laguerre and other classical special functions.  For example, sampling the sine function at 100 points between $0$ and $2\pi$ is as simple as:

In [ ]:
x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)

## Basic Linear Algebra

Traditional matrix operations can be accessed from the numpy.linalg module (basically a wrapper around LAPACK).

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[2, 3], [4, 5]])

In [ ]:
a.dot(b)

In [ ]:
np.linalg.eig(a)

In [ ]:
np.linalg.inv(b)

## An example from high school chemistry: stoichiometry

Let's find the coefficients that balance the following simple oxidation reaction:

$$\textrm{aCH}_4 + \textrm{bO}_2 \rightarrow \textrm{cCO}_2 + \textrm{dH}_2\textrm{O}$$

we can represent this as the linear ssytem

$$Ax=y$$

with

$$
\begin{pmatrix}1 & 0 & -1 & 0\\
4 & 0 & 0 & -2\\
0 & 2 & -2 & -1\\
0 & 0 & 0 & 1
\end{pmatrix}\begin{pmatrix}a\\
b\\
c\\
d
\end{pmatrix}=\begin{pmatrix}0\\
0\\
0\\
1
\end{pmatrix}
$$

so we find our coefficients $(a, b, c, d)$ by solving for $x$:

In [ ]:
A = np.array([[1,0,-1,0], [4,0,0,-2] , [0,2,-2,-1], [0,0,0,1]])
y = np.array([0, 0, 0, 1])
x = np.linalg.solve(A, y)
print('a, b, c, d:')
print(x / x.min())  # so the smallest coefficients are 1

In [ ]:
from IPython.display import Markdown as md

a, b, c, d = (x / x.min()).astype(int)

result = f"""
Or in classical chemical notation:

$${a}\\textrm{{CH}}_4 + {b} \\textrm{{O}}_2 \\rightarrow {c} \\textrm{{CO}}_2 + 
{d} \\textrm{{H}}_2\\textrm{{O}}$$
"""
md(result)

## Masked Arrays

MaskedArrays are a subclass of ndarrays containing a Boolean mask to indicate invalid data.

In [ ]:
x = np.array([1, 2, 3, -1, 5])

In [ ]:
mx = np.ma.masked_array(x, mask=[0, 0, 0, 1, 0])

In [ ]:
mx.data, mx.mask

In [ ]:
mx.mean()

In [ ]:
x = np.ma.array([1, 2, 3])

In [ ]:
x[0] = np.ma.masked
x

In [ ]:
x = np.ma.array([-1, 1, 0, 2, 3], mask=[0, 0, 0, 0, 1])

In [ ]:
np.log(x)

## Stacking and concatenation

numpy also provides handy routines to concatenate (and split) arrays.

In [ ]:
a = np.array([[1, 2], [3, 4]])
a

In [ ]:
b = np.array([[5, 6]])
b

In [ ]:
print a.shape, b.shape

In [ ]:
x = np.concatenate((a, b), axis=0)

In [ ]:
print x, x.shape

In [ ]:
y = np.concatenate((a, b.T), axis=1)

In [ ]:
print y, y.shape

In [ ]:
np.vstack((a,b))

In [ ]:
np.hstack((a,b.T))

**Optional Exercise: reading a text file with mixed data**

Consider loading data from a text file with the format:

<pre>
# Station  Lat    Long   Elev 
BIRA	26.4840	87.2670	0.0120
BUNG	27.8771	85.8909	1.1910
etc...
</pre>

These are seismic measurement stations in the Himalaya, with the elevation indicated in km.  Data with a structure such as this is common in many disciplines, and because we have a combination of text and numerical fields, we can't directly load it into a numpy array with a simple dtype. We'll need to define a dtype that matches this kind of data.

Questions:

1. Define a dtype descriptor for the above fields.
1. Load the file `stations.txt` into an array called `stations` using `np.loadtxt` and the dtype you just created.
1. *Think:* what are the dimensions of the array you just created? Is it a 1-d array or a 2-d one?
1. Print the mean latitude, mean longitude and maximum elevation.
1. *Bonus:* find the name of the station with maximum elevation (hint: `np.argmax`)


In [ ]:
#%load soln_stations.py

**Note:** for tabular data, the Pandas library (covered later today) provides much more powerful and convenient facilities.  The following code would read the above file using Pandas:

    read_table('stations.txt', names=('station', 'lat', 'lon', 'elev'), skiprows=1)

But numpy's structured arrays can be used in many contexts where Pandas is not the appropriate tool, such as multidimensional arrays with complex dtypes. Imagine for example describing the temperature `T` and pressure `p` in a 3-d volume represented by a $2\times 3 \times 4$ array that you initialize to constant values for a simulation:

In [ ]:
x = np.zeros((2,3,4), dtype=[('T', float), ('p', float)])
x['T'] = 10
x['p'] = 20
x

We'll also learn about `xarray` later on in the course...